In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [3]:
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []
    
    positive_path = path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]
        
    negative_path = path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list += [negative_path + f]
    
    print('read', filetype, 'files:', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    return all_labels, all_texts   

In [4]:
y_train, train_text = read_files("train")

read train files: 25000


In [5]:
y_test,  test_text  = read_files("test")

read test files: 25000


In [6]:
token = Tokenizer(num_words = 3800)
token.fit_on_texts(train_text)

In [7]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [8]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 380)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen = 380)

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

In [10]:
model = Sequential()

In [11]:
model.add(Embedding(output_dim = 32,
                    input_dim  = 3800,
                    input_length = 380))
model.add(Dropout(0.2))

In [12]:
model.add(LSTM(32))

In [13]:
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.2))

In [14]:
model.add(Dense(units = 1, activation = 'sigmoid'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 138,625
Trainable params: 138,625
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [17]:
train_history = model.fit(x_train, 
                          y_train, 
                          batch_size = 100,
                          epochs = 10,
                          verbose = 2,
                          validation_split = 0.2
                         )

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 31s - loss: 0.4796 - acc: 0.7606 - val_loss: 0.6547 - val_acc: 0.7090
Epoch 2/10
 - 30s - loss: 0.2665 - acc: 0.8935 - val_loss: 0.4168 - val_acc: 0.8244
Epoch 3/10
 - 37s - loss: 0.2287 - acc: 0.9108 - val_loss: 0.3222 - val_acc: 0.8592
Epoch 4/10
 - 33s - loss: 0.1965 - acc: 0.9255 - val_loss: 0.2933 - val_acc: 0.8744
Epoch 5/10
 - 32s - loss: 0.1742 - acc: 0.9344 - val_loss: 0.3223 - val_acc: 0.8686
Epoch 6/10
 - 32s - loss: 0.1645 - acc: 0.9386 - val_loss: 0.5180 - val_acc: 0.8028
Epoch 7/10
 - 32s - loss: 0.1578 - acc: 0.9403 - val_loss: 0.4298 - val_acc: 0.8596
Epoch 8/10
 - 35s - loss: 0.1322 - acc: 0.9517 - val_loss: 0.5485 - val_acc: 0.8180
Epoch 9/10
 - 34s - loss: 0.1168 - acc: 0.9571 - val_loss: 0.5184 - val_acc: 0.8470
Epoch 10/10
 - 31s - loss: 0.1058 - acc: 0.9618 - val_loss: 0.5684 - val_acc: 0.8424


In [18]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 14s 543us/step


0.85516